In [1]:
# Imports

import pandas as pd
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, normalize
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Read training data
data = pd.read_csv("train.csv")

# Print length of training data
print(len(data))

# Print first 5 entries of df
data.head()

891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
print("Missing values per Column:\n")

for col in data.columns:
    print(f'{col}: {data[col].isnull().sum()}')

Missing values per Column:

PassengerId: 0
Survived: 0
Pclass: 0
Name: 0
Sex: 0
Age: 177
SibSp: 0
Parch: 0
Ticket: 0
Fare: 0
Cabin: 687
Embarked: 2


In [5]:
data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [6]:
## Feature engineering
# PassengerID is redundant
# Name is hard to be a predictor
# Ticket has 681/891 so probably not a predictor
# Cabin a lot of missing values

del data["PassengerId"]
del data["Name"]
del data["Ticket"]
del data["Cabin"]

In [7]:
# Replace male/female by 0 or 1 in order to be readable by ML model
# Age has a lot of missing values, replace age by avg
# Embarked has 2 missing values, replace by S as it occurs most often

data['Sex'].replace('female', 0, inplace=True)
data['Sex'].replace('male', 1, inplace=True)
data['Age'].fillna(data.Age.mean(), inplace=True)
data['Embarked'].fillna('S', inplace=True)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [8]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [9]:
# One-hot encode/make dummies for variable Embarked

one_hot = pd.get_dummies(data.Embarked, prefix='Embarked')
data = data.drop('Embarked',axis = 1)
data = data.join(one_hot)
one_hot_ = pd.get_dummies(data.Pclass, prefix='Pclass')
data = data.drop('Pclass',axis = 1)
data = data.join(one_hot_)
data.round(1)
data.fillna(0)

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,0,1,22.000000,1,0,7.2500,0,0,1,0,0,1
1,1,0,38.000000,1,0,71.2833,1,0,0,1,0,0
2,1,0,26.000000,0,0,7.9250,0,0,1,0,0,1
3,1,0,35.000000,1,0,53.1000,0,0,1,1,0,0
4,0,1,35.000000,0,0,8.0500,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,27.000000,0,0,13.0000,0,0,1,0,1,0
887,1,0,19.000000,0,0,30.0000,0,0,1,1,0,0
888,0,0,29.699118,1,2,23.4500,0,0,1,0,0,1
889,1,1,26.000000,0,0,30.0000,1,0,0,1,0,0


In [10]:
# Convert dataframe to numpy arrays for ML
# Target variable

y = data.Sex.to_numpy()
data = data.drop('Sex', axis = 1)
data.head()

,Survived,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,0,22.0,1,0,7.2500,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,1,0,0
2,1,26.0,0,0,7.9250,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,0,0,1,1,0,0
4,0,35.0,0,0,8.0500,0,0,1,0,0,1


In [11]:
X = data.to_numpy()

In [12]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
# Apply Machine learning (K-fold Crossvalidation)

names = ["Random Forest", "NaiveBayes", "Logistic Regression", "Neural Networks"]
classifiers = [RandomForestClassifier(n_estimators=5), GaussianNB(), LogisticRegression(random_state=0),
               MLPClassifier(solver='lbfgs', alpha=0.5, hidden_layer_sizes=(5, 2), random_state=1, max_iter=10000)]

for i, clf in enumerate(classifiers):
    scores = cross_val_score(clf, X, y, cv=5)
    print(f"{names[i]}: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Random Forest: 0.78 accuracy with a standard deviation of 0.03
NaiveBayes: 0.75 accuracy with a standard deviation of 0.03
Logistic Regression: 0.78 accuracy with a standard deviation of 0.02


/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

Neural Networks: 0.80 accuracy with a standard deviation of 0.01
